In [8]:
import cv2
import mediapipe as mp
import math

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1 = landmark1.x, landmark1.y
    x2, y2 = landmark2.x, landmark2.y
    x3, y3 = landmark3.x, landmark3.y
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    return angle

def classifyPose(landmarks):
    left_shoulder = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    left_elbow = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
    left_wrist = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

    right_shoulder = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    right_elbow = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
    right_wrist = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

    left_hip = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
    left_knee = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
    right_knee = landmarks.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]

    # Ensure all landmarks are detected
    if (left_shoulder and left_elbow and left_wrist and
        right_shoulder and right_elbow and right_wrist and
        left_hip and right_hip and left_knee and right_knee):

        left_elbow_angle = calculateAngle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculateAngle(right_shoulder, right_elbow, right_wrist)

        left_shoulder_angle = calculateAngle(left_elbow, left_shoulder, left_hip)
        right_shoulder_angle = calculateAngle(right_elbow, right_shoulder, right_hip)

        left_knee_angle = calculateAngle(left_hip, left_knee, left_wrist)
        right_knee_angle = calculateAngle(right_hip, right_knee, right_wrist)

        # Classify as "Warrior II Pose" if all conditions are met
        if (165 < left_elbow_angle < 195 and 165 < right_elbow_angle < 195 and
            80 < left_shoulder_angle < 110 and 80 < right_shoulder_angle < 110 and
            165 < left_knee_angle < 195 and 165 < right_knee_angle < 195):
            return 'Warrior II Pose'

    return 'Unknown Pose'

pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

video = cv2.VideoCapture(0)
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

while video.isOpened():
    ok, frame = video.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_video.process(rgb_frame)

    if results.pose_landmarks:
        pose_label = classifyPose(results)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.putText(frame, pose_label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)

    cv2.imshow('Pose Detection', frame)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()
